<a href="https://colab.research.google.com/github/juanmrd/juanmrd.github.io/blob/main/IncicenciasFinal.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Importar Librerias
import pandas as pd
import numpy as np

from google.colab import drive

# Montar Google Drive
#drive.mount('/content/drive')

# Ruta del archivo en Google Drive
ruta_archivo = '/content/incidencias.csv'

# Leer el archivo de Excel en un DataFrame de Pandas
incidencia = pd.read_csv(ruta_archivo, delimiter=';')

# Definir la función para aplicar estilos condicionales
def resaltar_filas(data):
    estilo = pd.DataFrame('', index=data.index, columns=data.columns)
    estilo.loc[data['Sala'].isin(['SALA1FIC', 'SAFIC01', 'SATICF01', 'SALA3FIC','SALA7FIC', 'SALA4FIC','SALA2FIC']), :] = 'background-color: turquoise'
    estilo.loc[data['Sala'].isin(['SA-EX-CC-ENF', 'SA-EX-CC-TEC', 'SA-EX-CC-FON', 'SA-EX-CC-KIN', 'SA-EX-CP-MED' ,'SA-EX-CC-PSI', 'SA-EX-CC-MED', 'PIF13', 'PIF14']), :] = 'background-color: yellow'
    return estilo

# Aplicar el estilo condicional al DataFrame
incidencia_styled = incidencia.style.apply(resaltar_filas, axis=None)

# Mostrar el DataFrame con los estilos aplicados
display(incidencia_styled)

# Lista de columnas a eliminar
columnas_a_eliminar = ['Usuario', 'Asistencia Pasada', 'Recuperada', 'Fecha Log.', 'Atrasado',
                       'Hr. Ped.', 'Minutos Atraso', 'Minutos Atraso Entrada', 'Minutos Atraso Salida',
                       'Min. Reales Trabajadas', 'Hrs. Reales Trabajadas', 'Centro de Negocio',
                       'Sigla Reg.', 'Valor Hora', 'Valor Clase', 'Observación', 'Sede',
                       'Hrs. Contratadas', 'Año', 'Periodo', 'Hrs. Ayudantía', 'Minutos',
                       'Hrs. Teóricas', 'Tipo Docente', 'Jornada']

# Eliminar las columnas
incidencia = incidencia.drop(columnas_a_eliminar, axis=1)

# Crear una nueva condición para las columnas 'Hrs. Log.' y 'Hora salida' vacías
nueva_condicion = incidencia['Hrs. Log.'].isna() & incidencia['Hora salida'].isna()

# Crear una condición para las nuevas condiciones
nuevas_condiciones = incidencia['Sala'].isin(['SALA1FIC', 'SATICF01', 'SALA2FIC', 'SALA4FIC', 'SALA7FIC','SALA3FIC', 'SAFIC01'])

# Crear una condición combinada para las condiciones originales y las nuevas
condiciones = (incidencia['Sala'].isin(['SA-EX-CC-ENF', 'SA-EX-CC-MED', 'SA-EX-CC-FON', 'SA-EX-CC-TEC', 'SA-EX-CC-KIN', 'SA-EX-CP-MED' , 'SA-EX-CC-PSI', 'PIF13', 'PIF14'])) | nuevas_condiciones

# Asignar 'REVISAR' a las filas que cumplen con la nueva condición
incidencia.loc[nueva_condicion, 'Incidencias'] = 'HORARIO INTERMEDIO'

# Asignar 'Campo Clínico' a las filas que cumplen con las condiciones originales
incidencia.loc[condiciones & ~nuevas_condiciones , 'Incidencias'] = 'Campo Clínico'

# Asignar 'Online' a las filas que cumplen con las nuevas condiciones
incidencia.loc[nuevas_condiciones, 'Incidencias'] = 'Online'

# Ordenar primero por fecha
incidencia = incidencia.sort_values(by='Fecha de la Clase')

# Definir la función para ordenar dentro de cada grupo de fecha
def ordenar_por_ramo_y_seccion(group):
    return group.sort_values(by=['Cod. Ramo', 'Cod. Sección'])

# Aplicar la función a cada grupo de fecha
incidencia = incidencia.groupby('Fecha de la Clase').apply(ordenar_por_ramo_y_seccion).reset_index(drop=True)

# Iterar sobre cada grupo de fecha
for Fecha, grupo in incidencia.groupby('Fecha de la Clase'):
    # Aplicar estilos condicionales al grupo
    grupo_styled = grupo.style.apply(resaltar_filas, axis=None)

    # Exportar el grupo a un archivo Excel
    nombre_archivo = f'incidencia_{Fecha}.xlsx'  # Nombre del archivo con la fecha en el nombre
    grupo_styled.to_excel(nombre_archivo, index=False)

<ipython-input-2-dfb3e6856289>:66: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  incidencia = incidencia.groupby('Fecha de la Clase').apply(ordenar_por_ramo_y_seccion).reset_index(drop=True)
